## Dependencias

In [ ]:
import pandas as pd
import json #JSON = Java Script Object Notation
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,DATE,CHAR,DATETIME
import os

pd.set_option('display.max_columns',None)

## Credenciales

In [ ]:
creds = json.load(open('credenciales1.json','rb'))

## Crear conexión a base de datos

In [ ]:
url = f"mysql+pymysql://{creds['user']}:{creds['password']}@{creds['servidor']}/netflix"
cnx = create_engine(url,encoding='utf8')
cnx = cnx.connect()

In [ ]:
cnx.closed

False

## Lectura y limpieza de datos

In [ ]:
datos = pd.read_csv('netflix_titles.csv')

In [ ]:
datos.shape

(8807, 12)

In [ ]:
datos.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


## Separación de Entidades

### Entidas show

In [ ]:
datos.columns.values

array(['show_id', 'type', 'title', 'director', 'cast', 'country',
       'date_added', 'release_year', 'rating', 'duration', 'listed_in',
       'description'], dtype=object)

In [ ]:
entShow = datos[['show_id', 'type', 'title', 'director', 'cast', 'country', 'date_added', 'release_year', 
                     'rating', 'duration', 'listed_in', 'description']].drop_duplicates().dropna().reset_index(drop=True)
entShow['show_id'] = entShow['show_id'].map(str.upper)
entShow.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,S8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...","September 24, 2021",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s..."
1,S9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,"September 24, 2021",2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...
2,S10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,"September 24, 2021",2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...


In [ ]:
aux = entShow['date_added'].str.split(expand=True)
aux.columns=['mes', 'dia', 'año']
aux['dia']=aux['dia'].replace({',':''}, regex=True)
dic_mes = {'September':'09', 'August':'08', 'July':'07', 'June':'06', 'May':'05', 'April':'04', 'March':'03',
           'February':'02', 'January':'01', 'December':'12', 'November':'11', 'October':'10'}
aux['mes']=aux['mes'].map(dic_mes)
aux['date_added_c']=aux.año + '-' + aux.mes + '-' + aux.dia
aux['date_added_c']=pd.to_datetime(aux['date_added_c'], format="%Y/%m/%d")
aux.head()

,mes,dia,año,date_added_c
0,09,24,2021,2021-09-24
1,09,24,2021,2021-09-24
2,09,24,2021,2021-09-24
3,09,23,2021,2021-09-23
4,09,21,2021,2021-09-21


In [ ]:
aux['mes'].unique()

array(['09', '08', '07', '06', '05', '04', '03', '02', '01', '12', '11',
       '10'], dtype=object)

In [ ]:
entShow['description'].str.len().max()

248

In [ ]:
entShow= pd.concat([entShow, aux['date_added_c']], axis=1)
entShow = entShow.drop(['date_added'], axis=1)
entShow.head()

,show_id,type,title,director,cast,country,release_year,rating,duration,listed_in,description,date_added_c
0,S8,Movie,Sankofa,Haile Gerima,"Kofi Ghanaba, Oyafunmike Ogunlano, Alexandra D...","United States, Ghana, Burkina Faso, United Kin...",1993,TV-MA,125 min,"Dramas, Independent Movies, International Movies","On a photo shoot in Ghana, an American model s...",2021-09-24
1,S9,TV Show,The Great British Baking Show,Andy Devonshire,"Mel Giedroyc, Sue Perkins, Mary Berry, Paul Ho...",United Kingdom,2021,TV-14,9 Seasons,"British TV Shows, Reality TV",A talented batch of amateur bakers face off in...,2021-09-24
2,S10,Movie,The Starling,Theodore Melfi,"Melissa McCarthy, Chris O'Dowd, Kevin Kline, T...",United States,2021,PG-13,104 min,"Comedies, Dramas",A woman adjusting to life after a loss contend...,2021-09-24
3,S13,Movie,Je Suis Karl,Christian Schwochow,"Luna Wedler, Jannis Niewöhner, Milan Peschel, ...","Germany, Czech Republic",2021,TV-MA,127 min,"Dramas, International Movies",After most of her family is murdered in a terr...,2021-09-23
4,S25,Movie,Jeans,S. Shankar,"Prashanth, Aishwarya Rai Bachchan, Sri Lakshmi...",India,1998,TV-14,166 min,"Comedies, International Movies, Romantic Movies",When the father of the man she loves insists t...,2021-09-21


In [ ]:
entShow.to_sql(con=cnx,
               name='netflix',
               if_exists='replace',
               index=False,
               chunksize=10000,
               dtype={'show_id': VARCHAR(5), 
                      'type': VARCHAR(7), 
                      'title': VARCHAR(104), 
                      'director': VARCHAR(208), 
                      'cast': VARCHAR(632), 
                      'country':VARCHAR(83),
                      'date_added_c':DATETIME, 
                      'release_year': INTEGER, 
                      'rating':VARCHAR(8), 
                      'duration':VARCHAR(10), 
                      'listed_in':VARCHAR(70),
                      'description':VARCHAR(248)})

5332